In [ ]:
!pip install transformers
!pip install contractions

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import keras
import pickle
import time
import datetime
import torch
import copy

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Embedding, GRU, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import get_linear_schedule_with_warmup, AdamW

from utilities import *

sns.set_style('whitegrid');
sns.set_context('notebook', font_scale=1.1);

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
root = '/content/drive/My Drive/surely_not_sarcasm_data/'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# prepare data

In [6]:
df = pd.read_csv(root+'data/train_balanced_smaller.csv', usecols=['label', 'comment'])
df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [7]:
X = df['comment'].values
y = df['label'].values

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, stratify=y_train, test_size=0.05, random_state=42)

In [9]:
X_train.shape, X_val.shape, X_test.shape

((768187,), (202155,), (40431,))

# distilbert

In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [11]:
# lengths = [len(tokenizer.encode(x, max_length=512, truncation=True)) for x in X_train]
# plt.hist(lengths);

In [ ]:
np.quantile(lengths, 0.9)

In [12]:
max_len = 25

In [13]:
encoding_train = tokenizer.batch_encode_plus(
    list(X_train),
    add_special_tokens=True,
    max_length=max_len,
    return_token_type_ids=False,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

encoding_val = tokenizer.batch_encode_plus(
    list(X_val),
    add_special_tokens=True,
    max_length=max_len,
    return_token_type_ids=False,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [14]:
X_train, X_train_mask = encoding_train['input_ids'], encoding_train['attention_mask']
X_val, X_val_mask = encoding_val['input_ids'], encoding_val['attention_mask']

In [15]:
y_train, y_val = torch.tensor(y_train), torch.tensor(y_val)

In [16]:
batch_size = 32

train_data = TensorDataset(X_train, X_train_mask, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(X_val, X_val_mask, y_val)
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [17]:
dataloaders = {'train': train_dataloader, 'val': val_dataloader}
data_size = {'train': len(train_data), 'val': len(val_data)}

In [18]:
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
).to(device);

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [19]:
epochs = 10
total_steps = len(train_dataloader) * epochs

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
best_model, history = train_model(model, criterion, optimizer, scheduler, epochs, dataloaders,
                                  data_size, device, 2, rep=3000)


======== Epoch 1 / 10 ========
Training...
Batch 3,000 of 24,006. Elapsed: 518.83 s.
Batch 6,000 of 24,006. Elapsed: 1037.63 s.
Batch 9,000 of 24,006. Elapsed: 1556.95 s.
Batch 12,000 of 24,006. Elapsed: 2076.65 s.
Batch 15,000 of 24,006. Elapsed: 2596.68 s.
Batch 18,000 of 24,006. Elapsed: 3116.46 s.
Batch 21,000 of 24,006. Elapsed: 3635.95 s.
Batch 24,000 of 24,006. Elapsed: 4156.03 s.
== train == loss: 0.5106 accuracy: 0.7461
== val == loss: 0.4832 accuracy: 0.7656

======== Epoch 2 / 10 ========
Training...
Batch 3,000 of 24,006. Elapsed: 519.96 s.
Batch 6,000 of 24,006. Elapsed: 1040.01 s.
Batch 9,000 of 24,006. Elapsed: 1560.07 s.
Batch 12,000 of 24,006. Elapsed: 2079.54 s.
Batch 15,000 of 24,006. Elapsed: 2599.97 s.
Batch 18,000 of 24,006. Elapsed: 3120.01 s.
Batch 21,000 of 24,006. Elapsed: 3640.32 s.
Batch 24,000 of 24,006. Elapsed: 4159.55 s.
== train == loss: 0.4500 accuracy: 0.7878
== val == loss: 0.4805 accuracy: 0.7687

======== Epoch 3 / 10 ========
Training...
Batch 3,

In [ ]:
torch.save(best_model.state_dict(), root+'model_distilbert.pth')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,4))

for i, which in enumerate(['accuracy', 'loss']):
    ax = axes[i]
    ax.plot(history['train_'+which], label='train_'+which);
    ax.plot(history['val_'+which], label='val_'+which);
    ax.set_xlabel('epoch');
    ax.set_ylabel(which);
    ax.legend();